## Setup Configs

In [ ]:
%%capture
! pip install -r Requirements.txt
! pip install kagglehub

In [84]:
# Importing Libraries   
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import kagglehub 
import os
import shutil

### First-time import sequence

In [ ]:
# Take the downloaded file from kagglehub and move it into a 'Data' folder in your current Directory.
test = os.curdir+'/Data'
os.makedirs(test,exist_ok=True)
path = kagglehub.dataset_download('zubairamuti/foodpanda-review-dataset')
filename= os.listdir(path)[0]
final_data_path = shutil.copy(path+f'/{filename}',test)

## Importing Data into environment and EDA

In [165]:
# Importing Data into Env
ds_new = pd.read_csv(final_data_path)
ds_new.head()

,customer_id,gender,age,city,signup_date,order_id,order_date,restaurant_name,dish_name,category,quantity,price,payment_method,order_frequency,last_order_date,loyalty_points,churned,rating,rating_date,delivery_status
0,C5663,Male,Adult,Peshawar,1/14/2024,O9663,8/23/2023,McDonald's,Burger,Italian,5,1478.27,Cash,38,7/19/2025,238,Active,3,10/14/2024,Cancelled
1,C2831,Male,Adult,Multan,7/7/2024,O6831,8/23/2023,KFC,Burger,Italian,3,956.04,Wallet,24,11/25/2024,81,Active,2,8/21/2025,Delayed
2,C2851,Other,Senior,Multan,6/20/2025,O6851,8/23/2023,Pizza Hut,Fries,Italian,2,882.51,Cash,42,5/10/2025,82,Inactive,3,9/19/2024,Delayed
3,C1694,Female,Senior,Peshawar,9/5/2023,O5694,8/23/2023,Subway,Pizza,Dessert,4,231.30,Card,27,7/24/2025,45,Inactive,2,6/29/2025,Delayed
4,C4339,Other,Senior,Lahore,12/29/2023,O8339,8/24/2023,KFC,Sandwich,Dessert,1,1156.69,Cash,35,12/21/2024,418,Inactive,3,3/6/2025,Cancelled


<span style="color:red;">From this dataset, I believe that these are insightful questions to ask:</span>
1. [What's the city with the biggest spending and what do they order](#question-1-whats-the-city-with-the-biggest-spending-and-what-do-they-order)
2. Are there any seasonal upticks in orders that we could capitalize off of? (Perhaps weekends or holidays)
3. Rate of Churn and whether average spend over a certain time period could indicate the likelihood of a certain customer churn (could use loyalty points as well - indicates how invested a customer is in our service)
4. A particular concern is if a lot of people are paying through cash (as it's hard to track and can't be controlled correctly), so how many people are paying through cash
5. Gender and age dynamics 
6. Average length of service before churn
7. Correlation between loyalty points and delivery_status? We have to ensure that atleast the loyal customers are getting good service

Before proceeding with this, let's check the quality of data and perform EDA to understand the data. 

In [166]:
ds_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      6000 non-null   object 
 1   gender           6000 non-null   object 
 2   age              6000 non-null   object 
 3   city             6000 non-null   object 
 4   signup_date      6000 non-null   object 
 5   order_id         6000 non-null   object 
 6   order_date       6000 non-null   object 
 7   restaurant_name  6000 non-null   object 
 8   dish_name        6000 non-null   object 
 9   category         6000 non-null   object 
 10  quantity         6000 non-null   int64  
 11  price            6000 non-null   float64
 12  payment_method   6000 non-null   object 
 13  order_frequency  6000 non-null   int64  
 14  last_order_date  6000 non-null   object 
 15  loyalty_points   6000 non-null   int64  
 16  churned          6000 non-null   object 
 17  rating        

Couple things to note here: 
1. age, gender, city, churned, payment_method, category, delivery_status will become $\underline{\text{categorical}}$ for classification/prediction purposes.
2. signup_date, rating_date, order_date, and last_order_date will be converted to $\underline{\text{datetime}}$ values
3. I will convert city to a $\underline{\text{string}}$ value

All these changes have been tabulated below:

|Column Name|Current Dtype|Converted Dtype|
|:---:|:---:|:---:| 
|age|object|categorical|
|gender|object|categorical|
|churned|object|categorical|
|payment_method|object|categorical|
|category|object|categorical|
|delivery_status|object|categorical|
|signup_date|object|datetime|
|rating_date|object|datetime|
|order_date|object|datetime|
|last_order_date|object|datetime|
|city|object|str|

We will convert datatypes now so that it's easier to work with down the line. For now, we move to some value counts. I opted for value counts as the plots don't visually show the difference in counts.

In [ ]:
# Value Counts for the categorical colum
col_names = ['age', 'gender','churned', 'payment_method']
for col in col_names:
    sub_df = ds_new[col].value_counts() # Get the 
    ds_new[col] = ds_new[col].astype('category')
    print(sub_df,'\n')

In [168]:
# Date columns
date_cols = ['signup_date', 'rating_date', 'order_date','last_order_date']
for col in date_cols:
    ds_new[col] = pd.to_datetime(ds_new[col])

In [ ]:
ds_new.info() # Final look

## Question 1: What's the city with the biggest spending and what do they order?

Now that we have done preliminary analysis of the data, let's move to answering our first question. As we only require city, price, and dish, we'll make a new dataset that only has those properties for easy manipulation. 

In [172]:
q1_features = ['city','price','dish_name']
q1_df = ds_new[q1_features]
q1_df.head()

,city,price,dish_name
0,Peshawar,1478.27,Burger
1,Multan,956.04,Burger
2,Multan,882.51,Fries
3,Peshawar,231.30,Pizza
4,Lahore,1156.69,Sandwich


Now, we use pandas' groupby function to group this data by city and find the subtotals

In [ ]:
q1_gb = (
    q1_df
    .groupby(by='city')['price']
    .sum()
    .sort_values(ascending=False)
)
print(f'The city with the biggest spending is {q1_gb.index[0]} with a total spend of ${q1_gb.values[0]}')
print(f'')

The city with the biggest spending is Multan with a total spend of $1009003.67


Examining this number in the ether, it seems like a lot, but we need to know the time period over which this spend has happened

In [196]:
def tdelta(df:pd.DataFrame,city:str)->int:
    latest_date = df.loc[df.city==city,'order_date'].max()
    oldest_date = df.loc[df.city==city,'order_date'].min()
    days_elapsed = (latest_date - oldest_date).days
    return days_elapsed